# Built-in Training and Evaluation Loops

Let us start for the most basic workflow.

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers

# create a model within a function to re-use it
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

# load data and reserve some part for validation
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss     ="sparse_categorical_crossentropy",
    metrics  =["accuracy"]
)
model.fit(
    train_images, train_labels,
    epochs=3,
    validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8623 - loss: 0.4506 - val_accuracy: 0.9588 - val_loss: 0.1407
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9526 - loss: 0.1638 - val_accuracy: 0.9682 - val_loss: 0.1144
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9643 - loss: 0.1257 - val_accuracy: 0.9725 - val_loss: 0.1009
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9692 - loss: 0.1024
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


There are a couple of ways you can customize this simple workflow:
1. Provide your own custom metrics.
2. Pass callbacks to the `fit()` method to schedule actions to be taken at specific points during training.

Commonly used metrics for classification and regression are already part of the built-in `keras.metrics` module, and most of the time that’s what you will use. But if you’re doing anything out of the ordinary, you will need to be able
to write your own metrics.

A Keras metric is a subclass of the `keras.metrics.Metric` class. Like layers, a metric has an internal state stored in TensorFlow variables. Unlike layers, these variables aren’t updated via backpropagation, so you have to write the state-update logic yourself, which happens in the `update_state()` method.

In [3]:
# Implementing a custom metric by subclassing the Metric class

import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    # Define the state variables in the constructor. Like for layers, you
    # have access to the add_weight() method.
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
        name="total_samples", initializer="zeros", dtype="int32")

    # Implement the state update logic in update_state(). The y_true argument
    # is the targets (or labels) for one batch, while y_pred represents the
    # corresponding predictions from the model.
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    # use the result() method to return the current value of the metric:
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    # expose a way to reset the metric state without having to reinstantiate it
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [4]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss     ="sparse_categorical_crossentropy",
    metrics  =["accuracy", RootMeanSquaredError()]
)
model.fit(
    train_images, train_labels,
    epochs=3,
    validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8635 - loss: 0.4483 - rmse: 0.4375 - val_accuracy: 0.9593 - val_loss: 0.1426 - val_rmse: 0.2490
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9534 - loss: 0.1586 - rmse: 0.2661 - val_accuracy: 0.9681 - val_loss: 0.1116 - val_rmse: 0.2186
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9635 - loss: 0.1316 - rmse: 0.2389 - val_accuracy: 0.9719 - val_loss: 0.1001 - val_rmse: 0.2075
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9694 - loss: 0.1103 - rmse: 0.2152
